# Calcul du nombre de prélèvements CVM non conforme par commune et par année

L'objectif de ce notebook est de partir de la liste de communes cog_communes, et pour chaque commune et chaque année, calculer le nombre de prélèvements non conformes pour le CVM.

Il y aura plusieurs aggrégations à faire :

- commune (inseecommune) peut avoir plusieurs UDIs (cdreseau) **ET** un UDI peut avoir plusieurs communes (inseecommune) 
- un prélèvement (referenceprel) peut être rattaché à plusieurs UDIs (cdreseau)
- un prélèvement (referenceprel) peut être composé de plusieurs paramètres (cdparametresiseeaux) ; mais dans le cas du CVM, il y a un seul paramètre selon la catégorisation de Pauline, donc c'est plus simple




In [1]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb
%config SqlMagic.displaylimit = 10

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

In [2]:
%%sql
SELECT * FROM int__lien_commune_cdreseau
LIMIT 5 

Running query in 'duckdb:///../../database/data.duckdb'

inseecommune,cdreseau,de_partition,nomcommune,quartiers,nomreseaux,debutalim
88498,088001424,2024,VECOUX,VECOUX,VECOUX,2010-08-17
88508,088001429,2024,VILLE-SUR-ILLON,VILLE-SUR-ILLON,VILLE-SUR-ILLON,2010-08-17
88513,088001431,2024,VINCEY,VINCEY,VINCEY,2010-08-17
88514,088001605,2024,VIOCOURT,VIOCOURT,RESEAU REMOVILLE,2010-08-17
88525,088002137,2024,VROVILLE,VROVILLE,RESEAU MELANGE VILLERS/AMBACOURT,2010-08-17


In [3]:
%%sql
SELECT * FROM int__lien_cdreseau_refreneceprel
LIMIT 5 

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,referenceprel,dateprel,heureprel,de_partition,datetimeprel
001000003,00100127677,2021-09-07,11h28,2021,2021-09-07 11:28:00
001000003,00100140209,2024-04-05,07h27,2024,2024-04-05 07:27:00
001000235,00100132896,2022-09-26,12h31,2022,2022-09-26 12:31:00
001000241,00100121967,2020-08-28,08h49,2020,2020-08-28 08:49:00
001000241,00100128646,2021-11-17,14h35,2021,2021-11-17 14:35:00


In [4]:
%%sql
SELECT * FROM int__resultats_udi_communes
LIMIT 5

Running query in 'duckdb:///../../database/data.duckdb'

referenceprel,cdparametresiseeaux,valtraduite,limitequal,de_partition,limitequal_float,unite,categorie,cdreseau,inseecommune,datetimeprel
00200179905,TCEYTCL,0.000,<=10 µg/L,2020,10.0,µg/L,hydrocarbures,002001064,02389,2020-05-19 11:48:00
00200179905,TCLEY,0.000,<=10 µg/L,2020,10.0,µg/L,hydrocarbures,002001064,02389,2020-05-19 11:48:00
00200179905,THM4,25.600,<=100 µg/L,2020,100.0,µg/L,sous produit désinfection,002001064,02389,2020-05-19 11:48:00
00200179905,TOL,0.000,None,2020,None,None,hydrocarbures,002001064,02389,2020-05-19 11:48:00
00200179905,XYLMEPA,0.000,None,2020,None,None,hydrocarbures,002001064,02389,2020-05-19 11:48:00


## Faire la liste complètes des communes (pour chaque catégorie et années)

In [5]:
%%sql --save list_communes
WITH
annees AS (
    SELECT unnest(generate_series(2020, 2024)) as annee
    ),

cat AS (
    SELECT categorie FROM int__mapping_category_simple GROUP BY 1 
    ),

year_cat AS (   
    SELECT
     annee, categorie
    FROM 
    annees
    CROSS JOIN
    cat 
)
    
SELECT DISTINCT
    year_cat.annee AS annee, 
    year_cat.categorie AS categorie,
    inseecommune ,
 FROM
      int__lien_commune_cdreseau
 FULL OUTER JOIN
       year_cat
 ON
    int__lien_commune_cdreseau.de_partition = year_cat.annee

Running query in 'duckdb:///../../database/data.duckdb'

annee,categorie,inseecommune
2020,nitrite,55385
2020,nitrite,56120
2020,nitrite,57003
2020,nitrite,57250
2020,nitrite,57425
2020,nitrite,58043
2020,nitrite,59118
2020,nitrite,59377
2020,nitrite,60209
2020,nitrite,61015


In [6]:
%%sql --with list_communes
SELECT
    *
FROM
   list_communes
WHERE
    inseecommune = '07194'
    AND categorie = 'cvm'

Running query in 'duckdb:///../../database/data.duckdb'

annee,categorie,inseecommune
2022,cvm,07194
2024,cvm,07194
2021,cvm,07194
2023,cvm,07194
2020,cvm,07194


## Test cog pour le final

In [7]:
%%sql 
SELECT 
    *
FROM 
  cog_communes
WHERE
    TYPECOM = 'COM'

Running query in 'duckdb:///../../database/data.duckdb'

TYPECOM,COM,REG,DEP,CTCD,ARR,TNCC,NCC,NCCENR,LIBELLE,CAN,COMPARENT,de_partition,de_ingestion_date,de_dataset_datetime
COM,01001,84,01,01D,012,5,ABERGEMENT CLEMENCIAT,Abergement-Clémenciat,L'Abergement-Clémenciat,0108,None,2024,2025-03-16,20240220
COM,01002,84,01,01D,011,5,ABERGEMENT DE VAREY,Abergement-de-Varey,L'Abergement-de-Varey,0101,None,2024,2025-03-16,20240220
COM,01004,84,01,01D,011,1,AMBERIEU EN BUGEY,Ambérieu-en-Bugey,Ambérieu-en-Bugey,0101,None,2024,2025-03-16,20240220
COM,01005,84,01,01D,012,1,AMBERIEUX EN DOMBES,Ambérieux-en-Dombes,Ambérieux-en-Dombes,0122,None,2024,2025-03-16,20240220
COM,01006,84,01,01D,011,1,AMBLEON,Ambléon,Ambléon,0104,None,2024,2025-03-16,20240220
COM,01007,84,01,01D,011,1,AMBRONAY,Ambronay,Ambronay,0101,None,2024,2025-03-16,20240220
COM,01008,84,01,01D,011,1,AMBUTRIX,Ambutrix,Ambutrix,0101,None,2024,2025-03-16,20240220
COM,01009,84,01,01D,011,1,ANDERT ET CONDON,Andert-et-Condon,Andert-et-Condon,0104,None,2024,2025-03-16,20240220
COM,01010,84,01,01D,011,1,ANGLEFORT,Anglefort,Anglefort,0110,None,2024,2025-03-16,20240220
COM,01011,84,01,01D,014,1,APREMONT,Apremont,Apremont,0114,None,2024,2025-03-16,20240220


In [8]:
%%sql 
WITH
cog AS (
    SELECT 
      DEP AS code_departement,
      REG AS code_region,
      COM AS inseecommune,
      LIBELLE AS name_commune,
    FROM 
      cog_communes
    WHERE
        TYPECOM = 'COM'
)
    
SELECT
        inseecommune,
        COUNT(code_departement) AS nb_code_departement,
        COUNT(code_region) AS nb_code_region,
        COUNT(name_commune) AS nb_name_commune,
FROM
    cog
GROUP BY 
       inseecommune
HAVING 
    nb_code_departement > 1 OR nb_code_region >1 OR nb_name_commune>1

Running query in 'duckdb:///../../database/data.duckdb'

inseecommune,nb_code_departement,nb_code_region,nb_name_commune


## Les résultats par communes - Bilan annuel

#### décomposition

In [9]:
%%sql --save communes_year
    SELECT 
     de_partition as annee,
     categorie,
     inseecommune,
     SUM(1) AS nb_analyses,
     SUM(case
            when limitequal_float is not NULL and valtraduite >= limitequal_float then 1
            else 0
        end ) AS nb_analyses_not_ok,
     SUM(case
            when limitequal_float is not NULL and valtraduite < limitequal_float then 1
            else 0
        end ) AS nb_analyses_ok,    
    FROM
      int__resultats_udi_communes
    GROUP BY 
        1,2,3

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

annee,categorie,inseecommune,nb_analyses,nb_analyses_not_ok,nb_analyses_ok
2020,microbio,54282,80,32,0
2020,microbio,57761,160,64,0
2020,microbio,59212,1233,436,0
2020,microbio,62229,60,24,0
2020,microbio,62738,40,16,0
2020,microbio,62769,105,42,0
2020,pesticides,63149,876,0,872
2020,pesticides,64227,2025,0,2025
2020,cvm,55211,2,0,2
2020,cvm,62704,7,0,7


In [10]:
%%sql --with communes_year --save int__resultats_communes
SELECT
    annee,
    inseecommune,
    categorie,
    coalesce(nb_analyses,0) AS nb_analyses,
    coalesce(nb_analyses_not_ok,0) AS nb_analyses_not_ok ,
    coalesce(nb_analyses_ok,0) AS nb_analyses_ok,
    CASE WHEN coalesce(nb_analyses,0) = 0 THEN 'Pas recherché'
         WHEN coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_ok,0) = 0 THEN 'jamais quantifié'
         WHEN categorie='cvm' AND coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_not_ok,0) > 1 THEN '> 0,5 µg/L'
         WHEN categorie='cvm' AND coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_ok,0) > 0 THEN '<= 0,5 µg/L'
         ELSE 'Not CVM - other'
    END AS resultat
FROM  
    communes_year
ORDER BY 
    annee

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

annee,inseecommune,categorie,nb_analyses,nb_analyses_not_ok,nb_analyses_ok,resultat
2020,54282,microbio,80,32,0,jamais quantifié
2020,57761,microbio,160,64,0,jamais quantifié
2020,59212,microbio,1233,436,0,jamais quantifié
2020,62229,microbio,60,24,0,jamais quantifié
2020,62738,microbio,40,16,0,jamais quantifié
2020,62769,microbio,105,42,0,jamais quantifié
2020,64227,pesticides,2025,0,2025,Not CVM - other
2020,55211,cvm,2,0,2,"<= 0,5 µg/L"
2020,60241,métaux lourds,9,0,9,Not CVM - other
2020,62704,cvm,7,0,7,"<= 0,5 µg/L"


###  SQL FINAL int__resultats_ALL_communes : Join avec list_communes_uid pour avoir la liste complète

#### Execution avec le SQL complet

In [11]:
%%sql --save int__resultats_ALL_communes
WITH 
/* list_communes_uid  */
annees AS (
    SELECT unnest(generate_series(2020, 2024)) as annee
    ),

cat AS (
    SELECT categorie FROM int__mapping_category_simple GROUP BY 1 
    ),

year_cat AS (   
    SELECT
     annee, categorie
    FROM 
    annees
    CROSS JOIN
    cat 
),

list_communes_uid AS (    
SELECT DISTINCT
    year_cat.annee AS annee, 
    year_cat.categorie AS categorie,
    inseecommune ,
 FROM
      int__lien_commune_cdreseau
 FULL OUTER JOIN
       year_cat
 ON
    int__lien_commune_cdreseau.de_partition = year_cat.annee
),
/* END  list_communes_uid  */


/* int__resultats_udi_communes */
    communes_year AS (
    SELECT 
     de_partition as annee,
     categorie,
     inseecommune,
     SUM(1) AS nb_analyses,
     SUM(case
            when limitequal_float is not NULL and valtraduite >= limitequal_float then 1
            else 0
        end ) AS nb_analyses_not_ok,
     SUM(case
            when limitequal_float is not NULL and valtraduite < limitequal_float then 1
            else 0
        end ) AS nb_analyses_ok,    
    FROM
      int__resultats_udi_communes
    GROUP BY 
        1,2,3
),

int__resultats_communes AS ( 
    SELECT
    annee,
    inseecommune,
    categorie,
    coalesce(nb_analyses,0) AS nb_analyses,
    coalesce(nb_analyses_not_ok,0) AS nb_analyses_not_ok ,
    coalesce(nb_analyses_ok,0) AS nb_analyses_ok,
    CASE WHEN coalesce(nb_analyses,0) = 0 THEN 'Pas recherché'
         WHEN coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_ok,0) = 0 THEN 'jamais quantifié'
         WHEN categorie='cvm' AND coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_not_ok,0) > 1 THEN '> 0,5 µg/L'
         WHEN categorie='cvm' AND coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_ok,0) > 0 THEN '<= 0,5 µg/L'
         ELSE 'Not CVM - other'
    END AS resultat
FROM  
    communes_year
ORDER BY 
    annee
    )
/* int__resultats_udi_communes END */
    
SELECT
    list_communes_uid.annee,
    list_communes_uid.inseecommune,
    list_communes_uid.categorie,
    coalesce(nb_analyses,0) AS nb_analyses,
    coalesce(nb_analyses_not_ok,0) AS nb_analyses_not_ok ,
    coalesce(nb_analyses_ok,0) AS nb_analyses_ok,
    coalesce(resultat,'Pas recherché') AS resultat
FROM  
 list_communes_uid
LEFT JOIN 
 int__resultats_communes
ON
    list_communes_uid.annee =  int__resultats_communes.annee
    AND list_communes_uid.categorie =  int__resultats_communes.categorie
    AND  list_communes_uid.inseecommune  =  int__resultats_communes.inseecommune


Your query is using one or more of the following snippets: int__resultats_communes, communes_year. JupySQL does not support snippet expansion within CTEs yet, CTE generation is disabled

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

annee,inseecommune,categorie,nb_analyses,nb_analyses_not_ok,nb_analyses_ok,resultat
2020,55385,nitrite,10,0,10,Not CVM - other
2020,56120,nitrite,255,0,255,Not CVM - other
2020,57003,nitrite,5,0,5,Not CVM - other
2020,57250,nitrite,5,0,5,Not CVM - other
2020,57425,nitrite,135,0,135,Not CVM - other
2020,58043,nitrite,15,1,14,Not CVM - other
2020,59118,nitrite,23,0,23,Not CVM - other
2020,59377,nitrite,18,0,18,Not CVM - other
2020,60209,nitrite,8,0,8,Not CVM - other
2020,61015,nitrite,80,0,80,Not CVM - other


In [12]:
%%sql --with int__resultats_ALL_communes
SELECT 
    *
FROM
    int__resultats_ALL_communes
WHERE
 int__resultats_ALL_communes.categorie = 'cvm'

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

annee,inseecommune,categorie,nb_analyses,nb_analyses_not_ok,nb_analyses_ok,resultat
2020,54018,cvm,7,0,7,"<= 0,5 µg/L"
2020,54027,cvm,2,0,2,"<= 0,5 µg/L"
2020,54054,cvm,2,0,2,"<= 0,5 µg/L"
2020,54071,cvm,3,0,3,"<= 0,5 µg/L"
2020,54113,cvm,4,0,4,"<= 0,5 µg/L"
2020,54153,cvm,3,0,3,"<= 0,5 µg/L"
2020,54154,cvm,1,0,1,"<= 0,5 µg/L"
2020,54164,cvm,1,0,1,"<= 0,5 µg/L"
2020,54175,cvm,3,0,3,"<= 0,5 µg/L"
2020,54183,cvm,3,0,3,"<= 0,5 µg/L"


###  Join avec cog pour avoir le nom des communes

In [13]:
%%sql --save int__resultats_ALL_communes_name
WITH 
/* list_communes_uid  */
annees AS (
    SELECT unnest(generate_series(2020, 2024)) as annee
    ),

cat AS (
    SELECT categorie FROM int__mapping_category_simple GROUP BY 1 
    ),

year_cat AS (   
    SELECT
     annee, categorie
    FROM 
    annees
    CROSS JOIN
    cat 
),

list_communes_uid AS (    
SELECT DISTINCT
    year_cat.annee AS annee, 
    year_cat.categorie AS categorie,
    inseecommune ,
 FROM
      int__lien_commune_cdreseau
 FULL OUTER JOIN
       year_cat
 ON
    int__lien_commune_cdreseau.de_partition = year_cat.annee
),
/* END  list_communes_uid  */


/* int__resultats_udi_communes */
    communes_year AS (
    SELECT 
     de_partition as annee,
     categorie,
     inseecommune,
     SUM(1) AS nb_analyses,
     SUM(case
            when limitequal_float is not NULL and valtraduite >= limitequal_float then 1
            else 0
        end ) AS nb_analyses_not_ok,
     SUM(case
            when limitequal_float is not NULL and valtraduite < limitequal_float then 1
            else 0
        end ) AS nb_analyses_ok,    
    FROM
      int__resultats_udi_communes
    GROUP BY 
        1,2,3
),

int__resultats_communes AS ( 
    SELECT
    annee,
    inseecommune,
    categorie,
    coalesce(nb_analyses,0) AS nb_analyses,
    coalesce(nb_analyses_not_ok,0) AS nb_analyses_not_ok ,
    coalesce(nb_analyses_ok,0) AS nb_analyses_ok,
    CASE WHEN coalesce(nb_analyses,0) = 0 THEN 'Pas recherché'
         WHEN coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_ok,0) = 0 THEN 'jamais quantifié'
         WHEN categorie='cvm' AND coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_not_ok,0) > 1 THEN '> 0,5 µg/L'
         WHEN categorie='cvm' AND coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_ok,0) > 0 THEN '<= 0,5 µg/L'
         ELSE 'Not CVM - other'
    END AS resultat
FROM  
    communes_year
ORDER BY 
    annee
    ),
/* int__resultats_udi_communes END */


    
/* int__resultats_ALL_communes */
int__resultats_ALL_communes AS (
SELECT
    list_communes_uid.annee,
    list_communes_uid.inseecommune,
    list_communes_uid.categorie,
    coalesce(nb_analyses,0) AS nb_analyses,
    coalesce(nb_analyses_not_ok,0) AS nb_analyses_not_ok ,
    coalesce(nb_analyses_ok,0) AS nb_analyses_ok,
    coalesce(resultat,'Pas recherché') AS resultat
FROM  
 list_communes_uid
LEFT JOIN 
 int__resultats_communes
ON
    list_communes_uid.annee =  int__resultats_communes.annee
    AND list_communes_uid.categorie =  int__resultats_communes.categorie
    AND  list_communes_uid.inseecommune  =  int__resultats_communes.inseecommune
)
/* int__resultats_ALL_communes END */
    
SELECT
  int__resultats_ALL_communes.*,
  cog_communes.DEP AS  code_departement,
  cog_communes.REG AS  code_region,
  cog_communes.LIBELLE AS name_commune,
FROM
  int__resultats_ALL_communes
LEFT JOIN  
  cog_communes
ON 
  int__resultats_ALL_communes.inseecommune = cog_communes.COM
WHERE 
    cog_communes.TYPECOM= 'COM'

Your query is using one or more of the following snippets: int__resultats_ALL_communes, communes_year, int__resultats_communes. JupySQL does not support snippet expansion within CTEs yet, CTE generation is disabled

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

annee,inseecommune,categorie,nb_analyses,nb_analyses_not_ok,nb_analyses_ok,resultat,code_departement,code_region,name_commune
2020,55385,nitrite,10,0,10,Not CVM - other,55,44,Nixéville-Blercourt
2020,56120,nitrite,255,0,255,Not CVM - other,56,53,Locqueltas
2020,57003,nitrite,5,0,5,Not CVM - other,57,44,Abreschviller
2020,57250,nitrite,5,0,5,Not CVM - other,57,44,Goetzenbruck
2020,57425,nitrite,135,0,135,Not CVM - other,57,44,Luppy
2020,58043,nitrite,15,1,14,Not CVM - other,58,27,Bussy-la-Pesle
2020,59118,nitrite,23,0,23,Not CVM - other,59,32,Busigny
2020,59377,nitrite,18,0,18,Not CVM - other,59,32,Marcoing
2020,60209,nitrite,8,0,8,Not CVM - other,60,32,La Corne-en-Vexin
2020,61015,nitrite,80,0,80,Not CVM - other,61,28,Aunou-sur-Orne


In [14]:
%%sql --with int__resultats_ALL_communes_name
SELECT 
    *
FROM
    int__resultats_ALL_communes_name
WHERE
     categorie = 'cvm'

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

annee,inseecommune,categorie,nb_analyses,nb_analyses_not_ok,nb_analyses_ok,resultat,code_departement,code_region,name_commune
2020,54018,cvm,7,0,7,"<= 0,5 µg/L",54,44,Anoux
2020,54027,cvm,2,0,2,"<= 0,5 µg/L",54,44,Atton
2020,54054,cvm,2,0,2,"<= 0,5 µg/L",54,44,Bayon
2020,54071,cvm,3,0,3,"<= 0,5 µg/L",54,44,Bezange-la-Grande
2020,54113,cvm,4,0,4,"<= 0,5 µg/L",54,44,Champenoux
2020,54153,cvm,3,0,3,"<= 0,5 µg/L",54,44,Dampvitoux
2020,54154,cvm,1,0,1,"<= 0,5 µg/L",54,44,Deneuvre
2020,54164,cvm,1,0,1,"<= 0,5 µg/L",54,44,Dommarie-Eulmont
2020,54175,cvm,3,0,3,"<= 0,5 µg/L",54,44,Einvaux
2020,54183,cvm,3,0,3,"<= 0,5 µg/L",54,44,Essey-la-Côte


# Check

In [15]:
%%sql
SELECT
    de_partition as year,
    COUNT(DISTINCT inseecommune) AS nb_inseecommune
FROM 
 edc_communes
GROUP BY 
    1 
ORDER BY 
   1

Running query in 'duckdb:///../../database/data.duckdb'

year,nb_inseecommune
2020,34788
2021,34833
2022,34874
2023,34852
2024,34809


In [16]:
%%sql --with int__resultats_ALL_communes  
 SELECT 
    annee,
    COUNT(DISTINCT inseecommune) AS nb_inseecommune
 FROM
   int__resultats_ALL_communes
GROUP BY 
    1
ORDER BY 
   1

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

annee,nb_inseecommune
2020,34788
2021,34833
2022,34874
2023,34852
2024,34809


In [17]:
%%sql --with int__resultats_ALL_communes_name  
 SELECT 
    annee,
    COUNT(DISTINCT inseecommune) AS nb_inseecommune
 FROM
   int__resultats_ALL_communes_name
GROUP BY 
    1
ORDER BY 
   1

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

annee,nb_inseecommune
2020,34774
2021,34819
2022,34859
2023,34850
2024,34803


**-> On perd des communes avec le LEFt JOIn. Pourquoi ??**

#### commune_code_insee = '07194'

In [18]:
%%sql
 SELECT 
   *
 FROM
   int__resultats_udi_communes
WHERE
    inseecommune = '07194'
    AND categorie = 'cvm'
ORDER BY 
    de_partition

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

referenceprel,cdparametresiseeaux,valtraduite,limitequal,de_partition,limitequal_float,unite,categorie,cdreseau,inseecommune,datetimeprel
00700166838,CLVYL,0.000,<=0.5 µg/L,2020,0.5,µg/L,cvm,007001617,07194,2020-02-05 13:52:00
00700167733,CLVYL,0.000,<=0.5 µg/L,2020,0.5,µg/L,cvm,007001388,07194,2020-04-15 09:44:00
00700169098,CLVYL,0.000,<=0.5 µg/L,2020,0.5,µg/L,cvm,007001388,07194,2020-06-17 10:36:00
00700174219,CLVYL,0.000,<=0.5 µg/L,2021,0.5,µg/L,cvm,007001617,07194,2021-03-31 08:06:00
00700176432,CLVYL,0.000,<=0.5 µg/L,2021,0.5,µg/L,cvm,007001388,07194,2021-07-26 09:32:00
00700176524,CLVYL,0.000,<=0.5 µg/L,2021,0.5,µg/L,cvm,007001388,07194,2021-07-13 08:43:00
00700187059,CLVYL,3.600,<=0.5 µg/L,2022,0.5,µg/L,cvm,007001617,07194,2022-08-31 13:31:00
00700187150,CLVYL,1.600,<=0.5 µg/L,2022,0.5,µg/L,cvm,007001617,07194,2022-09-14 14:29:00
00700187169,CLVYL,0.370,<=0.5 µg/L,2022,0.5,µg/L,cvm,007001617,07194,2022-09-21 09:20:00
00700187170,CLVYL,1.200,<=0.5 µg/L,2022,0.5,µg/L,cvm,007001617,07194,2022-09-21 09:05:00


In [19]:
%%sql --with int__resultats_communes 
 SELECT 
   *
 FROM
   int__resultats_communes
WHERE
    inseecommune = '07194'
    AND categorie = 'cvm'
ORDER BY 
    annee

Running query in 'duckdb:///../../database/data.duckdb'

annee,inseecommune,categorie,nb_analyses,nb_analyses_not_ok,nb_analyses_ok,resultat
2020,07194,cvm,3,0,3,"<= 0,5 µg/L"
2021,07194,cvm,3,0,3,"<= 0,5 µg/L"
2022,07194,cvm,11,8,3,"> 0,5 µg/L"
2023,07194,cvm,16,11,5,"> 0,5 µg/L"
2024,07194,cvm,10,5,5,"> 0,5 µg/L"


In [20]:
%%sql --with int__resultats_ALL_communes_name 
 SELECT 
   *
 FROM
   int__resultats_ALL_communes_name
WHERE
    inseecommune = '07194'
    AND categorie = 'cvm'
ORDER BY 
    annee

Running query in 'duckdb:///../../database/data.duckdb'

annee,inseecommune,categorie,nb_analyses,nb_analyses_not_ok,nb_analyses_ok,resultat,code_departement,code_region,name_commune
2020,07194,cvm,3,0,3,"<= 0,5 µg/L",07,84,Rochessauve
2021,07194,cvm,3,0,3,"<= 0,5 µg/L",07,84,Rochessauve
2022,07194,cvm,11,8,3,"> 0,5 µg/L",07,84,Rochessauve
2023,07194,cvm,16,11,5,"> 0,5 µg/L",07,84,Rochessauve
2024,07194,cvm,10,5,5,"> 0,5 µg/L",07,84,Rochessauve


In [21]:
%%sql --with int__resultats_ALL_communes_name 
 SELECT 
   resultat, MIN(inseecommune)
 FROM
   int__resultats_ALL_communes_name
WHERE 
    categorie = 'cvm'
GROUP BY 1

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

resultat,min(inseecommune)
Pas recherché,01002
"> 0,5 µg/L",04018
"<= 0,5 µg/L",01001
jamais quantifié,08031


#### commune_code_insee = '01002'

In [22]:
%%sql 
 SELECT 
   *
 FROM
   int__resultats_udi_communes
WHERE
    inseecommune = '01002'
    AND categorie = 'cvm'
ORDER BY 
    de_partition

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

referenceprel,cdparametresiseeaux,valtraduite,limitequal,de_partition,limitequal_float,unite,categorie,cdreseau,inseecommune,datetimeprel
00100130490,CLVYL,0.000,<=0.5 µg/L,2022,0.5,µg/L,cvm,001000369,01002,2022-04-26 09:56:00
00100139937,CLVYL,0.000,<=0.5 µg/L,2024,0.5,µg/L,cvm,001000369,01002,2024-03-26 15:10:00


In [23]:
%%sql --with int__resultats_ALL_communes_name 
 SELECT 
   *
 FROM
   int__resultats_ALL_communes_name
WHERE
    inseecommune = '01002'
    AND categorie = 'cvm'
ORDER BY 
    annee

Running query in 'duckdb:///../../database/data.duckdb'

annee,inseecommune,categorie,nb_analyses,nb_analyses_not_ok,nb_analyses_ok,resultat,code_departement,code_region,name_commune
2020,01002,cvm,0,0,0,Pas recherché,01,84,L'Abergement-de-Varey
2021,01002,cvm,0,0,0,Pas recherché,01,84,L'Abergement-de-Varey
2022,01002,cvm,1,0,1,"<= 0,5 µg/L",01,84,L'Abergement-de-Varey
2023,01002,cvm,0,0,0,Pas recherché,01,84,L'Abergement-de-Varey
2024,01002,cvm,1,0,1,"<= 0,5 µg/L",01,84,L'Abergement-de-Varey


#### commune_code_insee = '08031'

In [24]:
%%sql 
 SELECT 
   *
 FROM
   int__resultats_udi_communes
WHERE
    inseecommune = '08031'
    AND categorie = 'cvm'
ORDER BY 
    de_partition

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

referenceprel,cdparametresiseeaux,valtraduite,limitequal,de_partition,limitequal_float,unite,categorie,cdreseau,inseecommune,datetimeprel
00800092483,CLVYL,0.700,<=0.5 µg/L,2020,0.5,µg/L,cvm,008000543,08031,2020-11-17 12:44:00
00800092855,CLVYL,0.000,<=0.5 µg/L,2020,0.5,µg/L,cvm,008000543,08031,2020-12-17 12:45:00
00800094118,CLVYL,0.000,<=0.5 µg/L,2021,0.5,µg/L,cvm,008000543,08031,2021-05-27 09:19:00
00800102498,CLVYL,0.000,<=0.5 µg/L,2023,0.5,µg/L,cvm,008000543,08031,2023-09-22 14:14:00
00800106109,CLVYL,0.900,<=0.5 µg/L,2024,0.5,µg/L,cvm,008000543,08031,2024-09-24 12:24:00
00800105054,CLVYL,0.800,<=0.5 µg/L,2024,0.5,µg/L,cvm,008000543,08031,2024-06-10 12:21:00


In [25]:
%%sql --with int__resultats_ALL_communes_name 
 SELECT 
   *
 FROM
   int__resultats_ALL_communes_name
WHERE
    inseecommune = '08031'
    AND categorie = 'cvm'
ORDER BY 
    annee

Running query in 'duckdb:///../../database/data.duckdb'

annee,inseecommune,categorie,nb_analyses,nb_analyses_not_ok,nb_analyses_ok,resultat,code_departement,code_region,name_commune
2020,08031,cvm,2,1,1,"<= 0,5 µg/L",08,44,Aure
2021,08031,cvm,1,0,1,"<= 0,5 µg/L",08,44,Aure
2022,08031,cvm,0,0,0,Pas recherché,08,44,Aure
2023,08031,cvm,1,0,1,"<= 0,5 µg/L",08,44,Aure
2024,08031,cvm,2,2,0,jamais quantifié,08,44,Aure
